In [14]:
import json 
import numpy as np
names = locals()
for i in range(9):
    emotion_label_path = 'emotion/recording_labels_process{}.json'.format(i)
    with open(emotion_label_path,'r', encoding='UTF-8') as f:
         names['emo_dict_' + str(i)] = json.load(f)

In [15]:
def get_emo_matrix(emo_dict_0,audio_name):
    key_ls = list(emo_dict_0[audio_name].keys())
    emo_label_ls = []
    for idx in key_ls:
    #     key_idx = (list(emo_dict_0[audio_name].keys())[idx])
        key_idx = idx
        start_time = float(key_idx.split(',')[0][1:])
        end_time = float(key_idx.split(',')[1][:-1])
        time_slot = end_time-start_time

        clip_emo_dict = emo_dict_0[audio_name][key_idx]
        # 'Surprise', 'Neutral', 'Sad', 'Happy', 'Angry'
        clip_emo_array = np.array(list(map(lambda x:float(x),clip_emo_dict.values())))
        emo_label_ls.append(clip_emo_array)
    if len(emo_label_ls)>1:
        emo_matrix = np.vstack(emo_label_ls)
    elif len(emo_label_ls)==1:
        emo_matrix = (emo_label_ls[0]).reshape((1,5))
    else:
        print('=========')
        print('ARRAY ERROR',audio_name,len(emo_label_ls))
        emo_matrix = None
#     print(emo_matrix.shape)
    return emo_matrix

In [16]:
emo_matrix_dict={}
for i in range(9):
    emo_dict = names['emo_dict_' + str(i)]
    for key in emo_dict.keys():
        new_emo_matrix = get_emo_matrix(emo_dict,key)
        if new_emo_matrix is not None:
            emo_matrix_dict[key] = new_emo_matrix

In [18]:
big_type_one_dict = np.load('big_type_one_dict.npy', allow_pickle=True).item()
big_type_ls = list(set(big_type_one_dict.values()))

single_clip_dict = emo_matrix_dict
ls_dict = {}
for clip_big in big_type_ls:
    ls_dict[clip_big]=[]
for key in single_clip_dict.keys():
    
    big_key = key[2:-4]
    
    if big_key in big_type_one_dict.keys():
        big_value = big_type_one_dict[big_key]
        ls_dict[big_value].append(big_key)
    else:
        print(big_key)
empty_ls = []
for key in ls_dict.keys():
    if len(ls_dict[key])<1:
        empty_ls.append(key)
        print(key)
for key in empty_ls:        
    ls_dict.pop(key)
    

In [20]:
def get_seq_matrix_dict():
    seq_dict = {}
    seq_matrix_dict = {}

    for clip_big in big_type_ls:
        seq_dict[clip_big]=[]

    for key in ls_dict.keys():
        for single_clip in ls_dict[key]:
            new_key = 'a_'+single_clip+'.wav'
            single_value = single_clip_dict[new_key]
            (seq_dict[key]).append(single_value)

    for key in seq_dict.keys():
        label_ls = seq_dict[key]
    #     print(label_ls)
        if len(label_ls)>1:
            label_matrix_value = np.vstack(label_ls)
            seq_matrix_dict[key] = label_matrix_value
    #         print(label_matrix_value.shape)
        elif len(label_ls)==1:
            label_matrix_value = label_ls[0]
            seq_matrix_dict[key] = label_matrix_value
    #         print(label_matrix_value.shape)
        elif len(label_ls)==0:
            print(key)
        else:
            print(key,len(label_ls))
    return seq_matrix_dict

In [21]:
def get_mean_dict(ori_dict):
    mean_dict = {}
    for key in list(ori_dict.keys()):
        mat = ori_dict[key]
        vec = np.mean(mat, axis=0)
        mean_dict[key] = vec
    return mean_dict

In [22]:
def get_speech_single_emo(speech_emo_dict):
    '''
    percent of different emo
    '''
    emo_class_dict = {}
    for video_id in speech_emo_dict.keys():
        emo_seq = speech_emo_dict[video_id]
        emo_class = []
        for k in range(len(emo_seq)):
            a = emo_seq[k]
            e_class = (np.where(a==np.max(a)))[0][0]
            emo_class.append(e_class)
        
        count_dict = {0:0,1:0,2:0,3:0,4:0}
        for key in emo_class:
            count_dict[key] = count_dict.get(key, 0) + 1
#         print(count_dict)
        
        p1 = (count_dict[0])/len(emo_class)
        p2 = (count_dict[1])/len(emo_class)
        p3 = (count_dict[2])/len(emo_class)
        p4 = (count_dict[3])/len(emo_class)
        p5 = (count_dict[4])/len(emo_class)
    
        emo_class_dict[video_id] = [p1,p2,p3,p4,p5]
    
    return emo_class_dict


In [24]:
speech_emo_dict = get_seq_matrix_dict()
speech_emo_mean_dict = get_mean_dict(speech_emo_dict)
emo_class_dict = get_speech_single_emo(speech_emo_dict)


np.save('speech_emo_dict.npy',speech_emo_dict)
np.save('speech_emo_mean_dict.npy',speech_emo_mean_dict)
np.save('emo_class_dict.npy',emo_class_dict)